# Erase Old Catalog
This notebook drops all user-defined functions, tables, and views from a catalog and then removes the schemas. Provide the catalog name with the widget below and run all cells.

In [ ]:

%pip install dbl-discoverx
dbutils.library.restartPython()


In [ ]:

dbutils.widgets.text("1.catalog", "old_catalog")

catalog = dbutils.widgets.get("1.catalog")


In [ ]:

skip_schemas = {"information_schema", "pg_catalog"}

from discoverx import DX

dx = DX()

schemas_df = spark.sql(f"SHOW SCHEMAS IN `{catalog}`")
schemas = [row[0] for row in schemas_df.collect() if row[0].lower() not in skip_schemas]

# Drop functions sequentially
for schema in schemas:
    funcs_df = spark.sql(f"SHOW USER FUNCTIONS IN `{catalog}`.`{schema}`")
    for f in funcs_df.collect():
        func_name = f[0]
        spark.sql(f"DROP FUNCTION IF EXISTS `{catalog}`.`{schema}`.`{func_name}`")

# Drop tables in parallel using DiscoverX
def drop_table(table_info):
    try:
        spark.sql(f"DROP TABLE IF EXISTS `{table_info.catalog}`.`{table_info.schema}`.`{table_info.table}`")
        return True
    except Exception as err:
        return str(err)

dx.from_tables(f"{catalog}.*.*").map(drop_table)

# Drop views and schemas sequentially
for schema in schemas:
    tables_df = spark.sql(f"SHOW TABLES IN `{catalog}`.`{schema}`")
    for t in tables_df.collect():
        desc = spark.sql(f"DESCRIBE EXTENDED `{catalog}`.`{schema}`.`{t.tableName}`").collect()
        t_type = next((row.data_type for row in desc if str(row.col_name).upper() == 'TYPE'), '').upper().replace('_', ' ')
        if t_type == 'VIEW':
            spark.sql(f"DROP VIEW IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`")
        elif t_type == 'MATERIALIZED_VIEW':
            spark.sql(f"DROP MATERIALIZED VIEW IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`")
    spark.sql(f"DROP SCHEMA IF EXISTS `{catalog}`.`{schema}`")


In [ ]:
print(f'Catalog `{catalog}` cleanup complete.')